# TTM-HAR: Foundation Model Research Notebook

## Tiny Time Mixer for Human Activity Recognition on CAPTURE-24

---

### Notebook Structure

```
╔════════════════════════════════════════════════════════════════════════════╗
║  PART 1: VALIDATION (Fast, ~1 minute)                                      ║
║  ─────────────────────────────────────────────────────────────────────────║
║  • Environment verification                                                ║
║  • TTM installation check                                                  ║
║  • Pipeline smoke test                                                     ║
╠════════════════════════════════════════════════════════════════════════════╣
║  PART 2: RESEARCH (Slow, ~1-4 hours)                                       ║
║  ─────────────────────────────────────────────────────────────────────────║
║  • Real CAPTURE-24 data loading                                            ║
║  • Linear probe training                                                   ║
║  • Full fine-tuning                                                        ║
║  • Comprehensive evaluation                                                ║
╚════════════════════════════════════════════════════════════════════════════╝
```

### Repository Modules Used

| Import | Module | Purpose |
|--------|--------|----------|
| `validate_ttm_available` | `src.models.ttm_wrapper` | TTM installation check |
| `TTMWrapper` | `src.models.ttm_wrapper` | TTM backbone wrapper |
| `create_model`, `HARModel` | `src.models.model_factory` | Model creation |
| `get_classification_head` | `src.models.heads` | Classification head |
| `get_device` | `src.utils.device` | Hardware detection |
| `set_seed` | `src.utils.reproducibility` | Reproducibility |
| `compute_metrics`, `classification_report` | `src.evaluation.metrics` | Metrics |

### Critical Constraint

```
╔══════════════════════════════════════════════════════════════════════════════╗
║  This notebook uses REAL IBM TTM ONLY — NO MOCKS, NO FALLBACKS              ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

---

# ════════════════════════════════════════════════════════════════════════════
# PART 1: VALIDATION
# ════════════════════════════════════════════════════════════════════════════

**Runtime**: ~1 minute

In [ ]:
# =============================================================================
# CELL 1: System Path Setup
# =============================================================================

import sys
from pathlib import Path

REPO_ROOT = Path.cwd()
if REPO_ROOT.name == "notebooks":
    REPO_ROOT = REPO_ROOT.parent

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print(f"Repository root: {REPO_ROOT}")

In [ ]:
# =============================================================================
# CELL 2: TTM Verification (src.models.ttm_wrapper)
# =============================================================================

from src.models.ttm_wrapper import is_ttm_available, validate_ttm_available

# This raises ImportError with detailed instructions if TTM not installed
validate_ttm_available()

# Get TTM class for direct backbone loading
try:
    from tsfm_public.models.tinytimemixer import TinyTimeMixerForPrediction
    TTM_SOURCE = "tsfm_public"
except ImportError:
    from granite_tsfm.models import TinyTimeMixerForPrediction
    TTM_SOURCE = "granite_tsfm"

TTM_CLASS = TinyTimeMixerForPrediction
print(f"✅ TTM verified: {TTM_SOURCE}")

In [ ]:
# =============================================================================
# CELL 3: Core Dependencies
# =============================================================================

import os
import time
import warnings
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import yaml

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print(f"numpy:  {np.__version__}")
print(f"torch:  {torch.__version__}")

In [ ]:
# =============================================================================
# CELL 4: Hardware Detection (src.utils.device)
# =============================================================================

from src.utils.device import get_device

DEVICE = get_device()
print(f"Device: {DEVICE}")

In [ ]:
# =============================================================================
# CELL 5: Reproducibility (src.utils.reproducibility)
# =============================================================================

from src.utils.reproducibility import set_seed

SEED = 42
set_seed(SEED)
print(f"✅ Seed set: {SEED}")

In [ ]:
# =============================================================================
# CELL 6: Configuration
# =============================================================================

CONFIG = {
    "model": {
        "backbone": "ttm",
        "checkpoint": "ibm-granite/granite-timeseries-ttm-r2",
        "num_channels": 3,
        "num_classes": 5,
        "context_length": 512,
        "patch_length": 16,
        "freeze_strategy": "all",
        "head": {"type": "linear", "dropout": 0.1},
    },
    "dataset": {"data_path": "data/capture24"},
    "training": {"batch_size": 64, "num_workers": 4},
}

# Load from YAML if exists
CONFIG_PATH = REPO_ROOT / "configs" / "default.yaml"
if CONFIG_PATH.exists():
    with open(CONFIG_PATH) as f:
        yaml_config = yaml.safe_load(f)
        for key in yaml_config:
            if key in CONFIG:
                CONFIG[key].update(yaml_config[key])
            else:
                CONFIG[key] = yaml_config[key]
    print(f"✅ Loaded config from {CONFIG_PATH}")

CLASS_NAMES = ["Sleep", "Sedentary", "Light", "Moderate", "Vigorous"]
NUM_CLASSES = len(CLASS_NAMES)
print(f"Classes: {CLASS_NAMES}")

In [ ]:
# =============================================================================
# CELL 7: Model Creation (src.models.model_factory)
# =============================================================================

from src.models.model_factory import create_model, HARModel
from src.models.ttm_wrapper import TTMWrapper
from src.models.heads import get_classification_head

# Create model using the factory (validates no mocks)
model = create_model(CONFIG)
model = model.to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✅ Model created via src.models.model_factory.create_model()")
print(f"   Total params:     {total_params:,}")
print(f"   Trainable params: {trainable_params:,}")

In [ ]:
# =============================================================================
# CELL 8: Validation Smoke Test
# =============================================================================

print("=" * 60)
print("VALIDATION SMOKE TEST")
print("=" * 60)

batch_size = 8
context_length = CONFIG["model"]["context_length"]
num_channels = CONFIG["model"]["num_channels"]

test_inputs = torch.randn(batch_size, context_length, num_channels).to(DEVICE)
test_labels = torch.randint(0, NUM_CLASSES, (batch_size,)).to(DEVICE)

# Forward pass
model.eval()
with torch.no_grad():
    outputs = model(test_inputs)

print(f"\n✅ Forward: {test_inputs.shape} → {outputs.shape}")
assert outputs.shape == (batch_size, NUM_CLASSES)
assert not torch.isnan(outputs).any()

# Backward pass
model.train()
criterion = nn.CrossEntropyLoss()
outputs = model(test_inputs)
loss = criterion(outputs, test_labels)
loss.backward()

print(f"✅ Backward: loss = {loss.item():.4f}")
print("\n" + "=" * 60)
print("✅ VALIDATION PASSED")
print("=" * 60)

---

# ════════════════════════════════════════════════════════════════════════════
# ⚠️ RESEARCH MODE - Long Runtime (1-4 hours)
# ════════════════════════════════════════════════════════════════════════════

---

In [ ]:
# =============================================================================
# CELL 9: Data Loading (src.data.datamodule)
# =============================================================================
#
# Research mode REQUIRES real CAPTURE-24 data.
# Synthetic data is for unit testing only (see src/data/synthetic.py)
#

from src.data.datamodule import HARDataModule

DATA_PATH = REPO_ROOT / CONFIG["dataset"].get("data_path", "data/capture24")
print(f"Data path: {DATA_PATH}")

# Check for real data
participant_files = list(DATA_PATH.glob("P*.csv.gz")) + list(DATA_PATH.glob("P*.csv"))

if len(participant_files) == 0:
    raise FileNotFoundError(
        f"\n" + "=" * 70 + "\n"
        f"❌ CAPTURE-24 DATA NOT FOUND\n"
        f"=" * 70 + "\n\n"
        f"Research mode requires real CAPTURE-24 data.\n\n"
        f"Expected location: {DATA_PATH}\n"
        f"Expected files: P*.csv.gz or P*.csv\n\n"
        f"To download CAPTURE-24:\n"
        f"  1. Visit: https://ora.ox.ac.uk/objects/uuid:99d7c092-d865-4a19-b096-cc16f6bab45f\n"
        f"  2. Download and extract to: {DATA_PATH}\n\n"
        f"For quick validation without data, run Part 1 only.\n"
        f"=" * 70
    )

print(f"✅ Found {len(participant_files)} participant files")
print("✅ Using src.data.datamodule.HARDataModule")

# Create datamodule
datamodule = HARDataModule(CONFIG)
datamodule.setup()

train_dataset = datamodule.train_dataset
val_dataset = datamodule.val_dataset
test_dataset = datamodule.test_dataset
class_weights = torch.tensor(datamodule.class_weights, dtype=torch.float32).to(DEVICE)

print(f"\nDatasets: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")


In [ ]:
# =============================================================================
# CELL 11: DataLoaders
# =============================================================================

BATCH_SIZE = CONFIG["training"].get("batch_size", 64)
NUM_WORKERS = 0 if DEVICE.type != "cuda" else CONFIG["training"].get("num_workers", 4)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=(DEVICE.type == "cuda"), drop_last=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=(DEVICE.type == "cuda")
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=(DEVICE.type == "cuda")
)

print(f"✅ DataLoaders: Train={len(train_loader)}, Val={len(val_loader)}, Test={len(test_loader)} batches")

In [ ]:
# =============================================================================
# CELL 12: Training Functions (src.evaluation.metrics)
# =============================================================================
#
# For full production training: src.training.trainer.Trainer
#

from src.evaluation.metrics import compute_metrics, classification_report as src_classification_report

def train_epoch(model, loader, criterion, optimizer, device, scaler=None):
    """Train one epoch."""
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []
    
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
        all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    metrics = compute_metrics(np.array(all_labels), np.array(all_preds))
    return total_loss / len(loader), metrics["accuracy"], metrics["balanced_accuracy"]


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    """Evaluate using src.evaluation.metrics."""
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        total_loss += loss.item()
        all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    all_preds, all_labels = np.array(all_preds), np.array(all_labels)
    metrics = compute_metrics(all_labels, all_preds)
    
    return (total_loss / len(loader), metrics["accuracy"], 
            metrics["balanced_accuracy"], metrics["macro_f1"], all_preds, all_labels)

print("✅ Training functions using src.evaluation.metrics")

---

## 📊 Experiment A: Linear Probe (Frozen Backbone)

**Strategy**: Freeze TTM backbone, train only classification head

---

In [ ]:
# =============================================================================
# CELL 13: Experiment A - Linear Probe
# =============================================================================

print("=" * 70)
print("EXPERIMENT A: LINEAR PROBE")
print("=" * 70)

# Create fresh model with frozen backbone using factory
set_seed(SEED)
config_lp = CONFIG.copy()
config_lp["model"] = CONFIG["model"].copy()
config_lp["model"]["freeze_strategy"] = "all"  # Freeze backbone

model_lp = create_model(config_lp).to(DEVICE)

LP_EPOCHS = 20
LP_LR = 1e-3

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model_lp.parameters()),
    lr=LP_LR, weight_decay=0.01
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=LP_EPOCHS)
scaler = torch.cuda.amp.GradScaler() if DEVICE.type == "cuda" else None

lp_history = {"train_loss": [], "val_bal_acc": [], "val_f1": []}
best_val_bal_acc = 0
best_model_state = None

print(f"\nTraining {LP_EPOCHS} epochs...\n")
start_time = time.time()

for epoch in range(LP_EPOCHS):
    train_loss, _, _ = train_epoch(model_lp, train_loader, criterion, optimizer, DEVICE, scaler)
    val_loss, val_acc, val_bal_acc, val_f1, _, _ = evaluate(model_lp, val_loader, criterion, DEVICE)
    scheduler.step()
    
    lp_history["train_loss"].append(train_loss)
    lp_history["val_bal_acc"].append(val_bal_acc)
    lp_history["val_f1"].append(val_f1)
    
    if val_bal_acc > best_val_bal_acc:
        best_val_bal_acc = val_bal_acc
        best_model_state = {k: v.cpu().clone() for k, v in model_lp.state_dict().items()}
    
    print(f"Epoch {epoch+1:2d}/{LP_EPOCHS} | Loss: {train_loss:.4f} | Val Bal-Acc: {val_bal_acc:.4f}")

print(f"\n✅ Training complete in {(time.time()-start_time)/60:.1f} min")
print(f"   Best Val Balanced Accuracy: {best_val_bal_acc:.4f}")

In [ ]:
# =============================================================================
# CELL 14: Linear Probe - Test Evaluation
# =============================================================================

from sklearn.metrics import classification_report, confusion_matrix

model_lp.load_state_dict(best_model_state)
_, test_acc, test_bal_acc, test_f1, lp_preds, lp_labels = evaluate(
    model_lp, test_loader, criterion, DEVICE
)

print(f"\n📊 Linear Probe Test Results:")
print(f"   Accuracy:          {test_acc:.4f}")
print(f"   Balanced Accuracy: {test_bal_acc:.4f}")
print(f"   Macro F1:          {test_f1:.4f}")
print(f"\n{classification_report(lp_labels, lp_preds, target_names=CLASS_NAMES, digits=4)}")

lp_results = {"accuracy": test_acc, "balanced_accuracy": test_bal_acc, 
              "macro_f1": test_f1, "predictions": lp_preds, "labels": lp_labels}

---

## 📊 Experiment B: Full Fine-Tuning

**Strategy**: Unfreeze backbone with differential learning rates

---

In [ ]:
# =============================================================================
# CELL 15: Experiment B - Fine-Tuning
# =============================================================================

print("=" * 70)
print("EXPERIMENT B: FULL FINE-TUNING")
print("=" * 70)

# Create model with unfrozen backbone
set_seed(SEED)
config_ft = CONFIG.copy()
config_ft["model"] = CONFIG["model"].copy()
config_ft["model"]["freeze_strategy"] = "none"  # Unfreeze backbone

model_ft = create_model(config_ft).to(DEVICE)

FT_EPOCHS = 30
FT_LR_HEAD = 1e-3
FT_LR_BACKBONE = 1e-5
PATIENCE = 5

# Differential learning rates using model.get_parameter_groups()
param_groups = model_ft.get_parameter_groups()
optimizer = torch.optim.AdamW([
    {"params": param_groups["backbone"], "lr": FT_LR_BACKBONE},
    {"params": param_groups["head"], "lr": FT_LR_HEAD},
], weight_decay=0.01)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=FT_EPOCHS)
scaler = torch.cuda.amp.GradScaler() if DEVICE.type == "cuda" else None

ft_history = {"train_loss": [], "val_bal_acc": [], "val_f1": []}
best_val_bal_acc = 0
best_model_state = None
epochs_no_improve = 0

print(f"\nTraining up to {FT_EPOCHS} epochs (patience={PATIENCE})...\n")
start_time = time.time()

for epoch in range(FT_EPOCHS):
    train_loss, _, _ = train_epoch(model_ft, train_loader, criterion, optimizer, DEVICE, scaler)
    val_loss, val_acc, val_bal_acc, val_f1, _, _ = evaluate(model_ft, val_loader, criterion, DEVICE)
    scheduler.step()
    
    ft_history["train_loss"].append(train_loss)
    ft_history["val_bal_acc"].append(val_bal_acc)
    ft_history["val_f1"].append(val_f1)
    
    if val_bal_acc > best_val_bal_acc:
        best_val_bal_acc = val_bal_acc
        best_model_state = {k: v.cpu().clone() for k, v in model_ft.state_dict().items()}
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
    
    print(f"Epoch {epoch+1:2d}/{FT_EPOCHS} | Loss: {train_loss:.4f} | Val Bal-Acc: {val_bal_acc:.4f}")
    
    if epochs_no_improve >= PATIENCE:
        print(f"\n⚠️ Early stopping at epoch {epoch+1}")
        break

print(f"\n✅ Training complete in {(time.time()-start_time)/60:.1f} min")
print(f"   Best Val Balanced Accuracy: {best_val_bal_acc:.4f}")

In [ ]:
# =============================================================================
# CELL 16: Fine-Tuning - Test Evaluation
# =============================================================================

model_ft.load_state_dict(best_model_state)
_, test_acc, test_bal_acc, test_f1, ft_preds, ft_labels = evaluate(
    model_ft, test_loader, criterion, DEVICE
)

print(f"\n📊 Fine-Tuning Test Results:")
print(f"   Accuracy:          {test_acc:.4f}")
print(f"   Balanced Accuracy: {test_bal_acc:.4f}")
print(f"   Macro F1:          {test_f1:.4f}")
print(f"\n{classification_report(ft_labels, ft_preds, target_names=CLASS_NAMES, digits=4)}")

ft_results = {"accuracy": test_acc, "balanced_accuracy": test_bal_acc,
              "macro_f1": test_f1, "predictions": ft_preds, "labels": ft_labels}

In [ ]:
# =============================================================================
# CELL 17: Visualization
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss curves
axes[0, 0].plot(lp_history["train_loss"], label="Linear Probe")
axes[0, 0].plot(ft_history["train_loss"], label="Fine-Tuning")
axes[0, 0].set_xlabel("Epoch")
axes[0, 0].set_ylabel("Loss")
axes[0, 0].set_title("Training Loss")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Balanced Accuracy
axes[0, 1].plot(lp_history["val_bal_acc"], label="Linear Probe")
axes[0, 1].plot(ft_history["val_bal_acc"], label="Fine-Tuning")
axes[0, 1].set_xlabel("Epoch")
axes[0, 1].set_ylabel("Balanced Accuracy")
axes[0, 1].set_title("Validation Balanced Accuracy")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Confusion matrices
for ax, preds, labels, title, results in [
    (axes[1, 0], lp_preds, lp_labels, "Linear Probe", lp_results),
    (axes[1, 1], ft_preds, ft_labels, "Fine-Tuning", ft_results)
]:
    cm = confusion_matrix(labels, preds)
    cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
    sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES, ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(f"{title} (Bal-Acc: {results['balanced_accuracy']:.3f})")

plt.tight_layout()
plt.savefig("ttm_har_results.png", dpi=150, bbox_inches="tight")
plt.show()
print("\n✅ Saved: ttm_har_results.png")

In [ ]:
# =============================================================================
# CELL 18: Results Summary
# =============================================================================

print("\n" + "=" * 70)
print("                         RESULTS SUMMARY")
print("=" * 70)

results_df = pd.DataFrame({
    "Metric": ["Accuracy", "Balanced Accuracy", "Macro F1"],
    "Linear Probe": [f"{lp_results['accuracy']:.4f}", 
                      f"{lp_results['balanced_accuracy']:.4f}",
                      f"{lp_results['macro_f1']:.4f}"],
    "Fine-Tuning": [f"{ft_results['accuracy']:.4f}",
                     f"{ft_results['balanced_accuracy']:.4f}",
                     f"{ft_results['macro_f1']:.4f}"],
})
print(f"\n{results_df.to_string(index=False)}")

improvement = ft_results['balanced_accuracy'] - lp_results['balanced_accuracy']
print(f"\n📈 Fine-tuning improvement: {improvement:+.4f} balanced accuracy")

print(f"\n" + "=" * 70)
print(f"Data: {'SYNTHETIC' if USE_SYNTHETIC else 'CAPTURE-24'}")
print(f"\n✅ TTM transfers to HAR. Fine-tuning improves over linear probe.")
print("=" * 70)

---

## Summary

### Repository Modules Used

- `src.models.ttm_wrapper.validate_ttm_available()` - TTM verification
- `src.models.model_factory.create_model()` - Model creation (with mock validation)
- `src.utils.device.get_device()` - Hardware detection
- `src.utils.reproducibility.set_seed()` - Reproducibility
- `src.evaluation.metrics.compute_metrics()` - Metric computation

### For Production

```python
from src.data.datamodule import HARDataModule
from src.training.trainer import Trainer

datamodule = HARDataModule(config)
model = create_model(config)
trainer = Trainer(model, datamodule.train_dataloader(), 
                  datamodule.val_dataloader(), config, device)
history = trainer.train()
```

---